In [ ]:
# Only need to do this once...
!pip install flask
!pip install --upgrade google-cloud-datastore


In [2]:
import json
import re
from flask import Flask, request, jsonify, make_response

from google.cloud import datastore

In [ ]:
app = Flask(__name__)
results = ""
rsize = 0
from random import seed
from random import randint
seed(1)

@app.route('/webhook/', methods=['POST'])
def handle():
    req = request.get_json(silent=True, force=True)
    print 'Request:'
    print(json.dumps(req, indent=4))
    topic = req.get('queryResult').get('parameters').get('topic')
    
    print topic
    # there could be multiple topics in a sentence
    if isinstance(topic, list):
        topic = topic[0]
    topic = re.sub(r'[^\w\s]', '', topic)
    topic = topic.lower()
    print("topic is ",topic)
    
    rsp = buildReply('Command not recognized')
    if (topic == "schedule"):
        rsp = scheduleMovie(req)
    elif (topic == "profile" or topic == "account"):
        rsp = getProfileAccount()
    elif (topic == "movie recommendation" ):
        rsp = getRecommendations()
        
    rsp = json.dumps(rsp, indent=4)
    print rsp
    r = make_response(rsp)
    r.headers['Content-Type'] = 'application/json'
    return r    
  
def scheduleMovie(req):
    confirm = req.get('queryResult').get('parameters').get('confirm').lower()
    print confirm
    if (confirm == "yes"):
        return buildReply('Sending request ...')
    return buildReply('Aborted')
  
def getProfileAccount():
    urls = ""
    with open('netflix_urls.txt') as netflix_urls:
        urls = netflix_urls.read()
    print urls
    return buildReply(urls)

def getRecommendations():
    global results 
    global rsize
    print "recom"
    if (rsize ==0):
        client = datastore.Client()
        query = client.query(kind='netflix')
        results = list(query.fetch(limit=100))
        rsize = len(results)
        print rsize
    return buildReply(results[randint(0, rsize -1 )]['title']+"\n"+results[randint(0, rsize-1)]['title'])
    
def getResponse(topic):
    
    client = datastore.Client()
    query = client.query(kind='Synonym')
    key = client.key('Synonym', topic)
    query.key_filter(key, '=')
    results = list(query.fetch())
    
    if len(results) == 0:
        return buildReply('I can\'t find that in the handbook...')
    
    print results[0]['synonym']
    
    query = client.query(kind='Topic')
    key = client.key('Topic', results[0]['synonym'])
    query.key_filter(key, '=')
    results = list(query.fetch())
    
    print results[0]['action_text']
    
    return buildReply(results[0]['action_text'])

def buildReply(info):
    return {
        'fulfillmentText': info,
    }

if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
[2021-03-14 03:19:57,856] ERROR in app: Exception on /webhook/ [POST]
Traceback (most recent call last):
  File "/usr/local/envs/py2env/lib/python2.7/site-packages/flask/app.py", line 1988, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/envs/py2env/lib/python2.7/site-packages/flask/app.py", line 1641, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/envs/py2env/lib/python2.7/site-packages/flask/app.py", line 1544, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/envs/py2env/lib/python2.7/site-packages/flask/app.py", line 1639, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/envs/py2env/lib/python2.7/site-packages/flask/app.py", line 1625, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-4-9df1e8cf9674>", line 19, in handle
    topic = re.sub(r'[^\w\s]'

Request:
{
    "queryResult": {
        "fulfillmentMessages": [
            {
                "text": {
                    "text": [
                        "Confirm watching movie 123 with friend James on 2021-03-14T12:00:00+11:00 at 2021-03-14T23:11:00+11:00 ?"
                    ]
                }
            }
        ], 
        "fulfillmentText": "Confirm watching movie 123 with friend James on 2021-03-14T12:00:00+11:00 at 2021-03-14T23:11:00+11:00 ?", 
        "allRequiredParamsPresent": true, 
        "parameters": {
            "date": "2021-03-14T12:00:00+11:00", 
            "movie": "123", 
            "scheduler": "", 
            "friend": {
                "name": "James"
            }, 
            "time": "2021-03-14T23:11:00+11:00"
        }, 
        "languageCode": "en", 
        "intentDetectionConfidence": 1.0, 
        "intent": {
            "displayName": "schedule-select-movie", 
            "name": "projects/i-backbone-307006/agent/intents/088680f2-046d-4

127.0.0.1 - - [14/Mar/2021 03:19:59] "POST /webhook/ HTTP/1.1" 200 -


Request:
{
    "queryResult": {
        "fulfillmentMessages": [
            {
                "text": {
                    "text": [
                        ""
                    ]
                }
            }
        ], 
        "allRequiredParamsPresent": true, 
        "parameters": {
            "topic": "schedule", 
            "confirm": "yes"
        }, 
        "languageCode": "en", 
        "intentDetectionConfidence": 1.0, 
        "intent": {
            "displayName": "schedule-movie-confirm", 
            "name": "projects/i-backbone-307006/agent/intents/e038dea7-7a57-47d5-8e09-98fa0833ee69"
        }, 
        "queryText": "yes", 
        "outputContexts": [
            {
                "parameters": {
                    "topic": "schedule", 
                    "confirm.original": "yes", 
                    "topic.original": "", 
                    "confirm": "yes", 
                    "movie": "123", 
                    "time": "2021-03-14T23:11:00+11:00", 


127.0.0.1 - - [14/Mar/2021 03:20:16] "POST /webhook/ HTTP/1.1" 200 -


...

Request:
{
    "queryResult": {
        "fulfillmentMessages": [
            {
                "text": {
                    "text": [
                        ""
                    ]
                }
            }
        ], 
        "allRequiredParamsPresent": true, 
        "parameters": {
            "topic": "schedule", 
            "confirm": "no"
        }, 
        "languageCode": "en", 
        "intentDetectionConfidence": 1.0, 
        "intent": {
            "displayName": "schedule-movie-confirm", 
            "name": "projects/i-backbone-307006/agent/intents/e038dea7-7a57-47d5-8e09-98fa0833ee69"
        }, 
        "queryText": "no", 
        "outputContexts": [
            {
                "parameters": {
                    "confirm.original": "no", 
                    "topic.original": "", 
                    "confirm": "no", 
                    "movie": "Firestorm", 
                    "time": "2021-03-14T15:00:00+11:00", 
                    "friend.original": "

[2021-03-14 03:32:50,136] ERROR in app: Exception on /webhook/ [POST]
Traceback (most recent call last):
  File "/usr/local/envs/py2env/lib/python2.7/site-packages/flask/app.py", line 1988, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/envs/py2env/lib/python2.7/site-packages/flask/app.py", line 1641, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/envs/py2env/lib/python2.7/site-packages/flask/app.py", line 1544, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/envs/py2env/lib/python2.7/site-packages/flask/app.py", line 1639, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/envs/py2env/lib/python2.7/site-packages/flask/app.py", line 1625, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-4-9df1e8cf9674>", line 19, in handle
    topic = re.sub(r'[^\w\s]', '', topic)
  File "/usr/local/envs/py2env/lib/python2.7/

Request:
{
    "queryResult": {
        "fulfillmentMessages": [
            {
                "text": {
                    "text": [
                        "Confirm watching movie firestorm with friend James on 2021-03-14T12:00:00+11:00 at 10:00:00 ?"
                    ]
                }
            }
        ], 
        "fulfillmentText": "Confirm watching movie firestorm with friend James on 2021-03-14T12:00:00+11:00 at 10:00:00 ?", 
        "allRequiredParamsPresent": true, 
        "parameters": {
            "date": "2021-03-14T12:00:00+11:00", 
            "movie": "firestorm", 
            "friend": {
                "name": "James"
            }, 
            "time": "2021-03-15T10:00:00+11:00"
        }, 
        "languageCode": "en", 
        "intentDetectionConfidence": 1.0, 
        "intent": {
            "displayName": "schedule-select-movie", 
            "name": "projects/i-backbone-307006/agent/intents/088680f2-046d-4c8a-bc7b-7e8759a6a2be"
        }, 
        "qu

127.0.0.1 - - [14/Mar/2021 03:32:53] "POST /webhook/ HTTP/1.1" 200 -


Request:
{
    "queryResult": {
        "fulfillmentMessages": [
            {
                "text": {
                    "text": [
                        ""
                    ]
                }
            }
        ], 
        "allRequiredParamsPresent": true, 
        "parameters": {
            "topic": "schedule", 
            "confirm": "yes"
        }, 
        "languageCode": "en", 
        "intentDetectionConfidence": 1.0, 
        "intent": {
            "displayName": "schedule-movie-confirm", 
            "name": "projects/i-backbone-307006/agent/intents/e038dea7-7a57-47d5-8e09-98fa0833ee69"
        }, 
        "queryText": "yes", 
        "outputContexts": [
            {
                "parameters": {
                    "confirm.original": "yes", 
                    "topic.original": "", 
                    "confirm": "yes", 
                    "movie": "firestorm", 
                    "time": "2021-03-15T10:00:00+11:00", 
                    "friend.original